### Scrapping INRIA- UOH morphological tags

In [1]:
# from bs4 import BeautifulSoup
# # import urllib2
# from urllib.request import urlopen
# import csv

# url = 'https://sanskritlibrary.org/helpmorphids.html'
# html = urlopen(url).read()
# soup = BeautifulSoup(html)
# tables = soup.findAll("table")

# with open("noun.csv", "w") as f:
#     wr = csv.writer(f)
#     wr.writerows([[td.text for td in row.find_all("td")] for row in tables[0].select("tr + tr")])

In [2]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate


In [3]:
# data.to_csv('DCS_to_UOH.csv',header=None,sep=',')
import pandas as pd
DCS_to_UOH = pd.read_csv('DCS_to_UOH.csv',sep=',',header=None)
dcs_list = list(DCS_to_UOH.iloc[:,4])
DCS_to_UOH.head()
# Create dcs to uoh dictionary
dcs_to_uoh_dict = dict()
for i in range(0,len(DCS_to_UOH)):
    dcs_to_uoh_dict[DCS_to_UOH.iloc[i,4]]= [i,DCS_to_UOH.iloc[i,5]]
map_unmapped_one = pd.read_csv('map_unmapped_one',sep=',',header=None)
map_unmapped_one_dict = dict()
for i in range(0,len(map_unmapped_one)):
    map_unmapped_one_dict[map_unmapped_one.iloc[i,0]] = map_unmapped_one.iloc[i,1]
map_unmapped_one.head()
uoh_rel_22_tags = pd.read_csv('uoh_rel_22_tags',sep=' ',header=None)
uoh_rel_22_tags_dict = dict()
for i in range(len(uoh_rel_22_tags)):
#     print(uoh_rel_22_tags[i,1].strip())
    uoh_rel_22_tags_dict[transliterate(uoh_rel_22_tags.iloc[i,1].strip(), sanscript.DEVANAGARI, sanscript.IAST)] = uoh_rel_22_tags.iloc[i,2]
# uoh_rel_22_tags_dict

### Get all 475 MA tags present in corpus

In [4]:
ma_set = set()
word = None
with open('pos_embedding_1_hot.475', 'r', encoding='utf-8') as f:
    # skip first line
    for i, line in enumerate(f):
        if i == 0:
            continue
        word, vec = line.split('@', 1)
        ma_set.add(word)

### Get freq of 475 tags

In [5]:
# ma_freq = dict()
# for m in ma_set:
#     ma_freq[m]= 0
# {k: v for k, v in sorted(ma_freq.items(), key=lambda item: item[1])}

# Get frequency of MA
# d=0
# paths = ['../Documents/Parsing/sanskrit_tree_bank/Preprocessed_amrith_data/Deliverables/1k_conll_sent_test/',\
#          '../Documents/Parsing/sanskrit_tree_bank/Preprocessed_amrith_data/Deliverables/2.5_conll_sent/']
# for path in paths:
#     for file in os.listdir(path):
#         df = pd.read_csv(path+file,sep=',')
#         d=d+1
#         for ma in df['morph']:
#             if ma == 'adv.':
#                 print()
# #             ma_freq[ma]=ma_freq[ma]+1
# print(d)

### Mapping corpus tags to UoH tags

In [6]:
def preprocess_tag(ma,dcs_list):
    ma =ma.replace('[','').replace(']','').replace('inst.','i.').replace('impft .','impft.')
    if ma in dcs_list:
        return ma
    elif '*' in ma:
        ma = ma.replace('*.','*')
        if ma.replace('*','m.') in dcs_list:
            return ma.replace('*','m.')
        elif ma.replace('*','n.') in dcs_list:
            return ma.replace('*','n.')
        else:
            return ma.replace('*','f.')
    else:
        return 'na'
            
def map_corpus_tag_to_dcs(ma):
    if preprocess_tag(ma,dcs_list) in dcs_list:
        return preprocess_tag(ma,dcs_list)
    elif ma in map_unmapped_one_dict.keys():
        if preprocess_tag(map_unmapped_one_dict[ma],dcs_list) in dcs_list:
            return preprocess_tag(map_unmapped_one_dict[ma],dcs_list)
    else:
        return 'na'

# ,map_unmapped_one_dict,dcs_list,dcs_to_uoh_dict

In [7]:
def match_Indeclineables(words,gt):
    count = 0
    for s in words:
        count = count + 1
        if '<' not in s or '>' not in s:
            print('UoH analysis in not available')
            continue
        elif 'vargaH' not in s:
            print('May be its verb')
            continue
        ma_temp =dict()
        for r in s.split('>'):
            if r:
                if len(r.split('<')) == 2:
                    ma_temp[r.split('<')[1].split(':')[0]] = r.split('<')[1].split(':')[1]
                else:
                    ma_temp[r.split(':')[0]] = r.split(':')[1]
            
        if ma_temp['vargaH'] == 'avy' and gt == 'avyaya':
            print('Predicted',ma_temp['vargaH'])
            print('Ground truth',gt)
            return count
           
    return -1
s = 'lap1<upasarga:Af><vargaH:avy><kqw_prawyayaH:lyap><XAwuH:lapaz><gaNaH:BvAxiH><level:2>'
match_Indeclineables([s],'avyaya')       


Predicted avy
Ground truth avyaya


1

In [8]:
def match_verb(words,gt):
    count = 0
    gt = gt.replace('{','').replace('}','').split(';')
    for s in words:
        count = count + 1  
        ma_temp =dict()
        if '<' not in s or '>' not in s:
            print('UoH analysis in not available')
            continue
        elif 'prayogaH' not in s and 'lakAraH' not in s:
            continue
        for r in s.split('>'):
            if r:
                if len(r.split('<')) == 2:
                    ma_temp[r.split('<')[1].split(':')[0]] = transliterate(r.split('<')[1].split(':')[1], sanscript.WX, sanscript.SLP1)
                else:
                    ma_temp[r.split(':')[0]] = transliterate(r.split(':')[1], sanscript.WX, sanscript.SLP1)
#         print(ma_temp)
        predicted = [ma_temp['prayogaH'],ma_temp['lakAraH'],ma_temp['puruRaH'],\
                                ma_temp['vacanam'],ma_temp['paxI']]
  
        
        if predicted == gt[:5]:
            print('Predicted',predicted)
            print('Ground truth',gt[:5])
            return count
    return -1

# match_verb(df.iloc[5,-1].split('/'),'{kartari;laN;pra;eka;parasmEpadI;;#;}')     

In [9]:
def match_noun(words,gt,ma):
    count = 0
    gt = gt.replace('}{',' ').strip('{').strip('}').replace(';',' ').split()
    if '*' in ma:
        gt[0] = 'a'
    for s in words:
        if '<' not in s or '>' not in s:
            print('UoH analysis in not available')
            continue
        elif 'lifgam' not in s and 'viBakwiH' not in s:
            print('It may be avyaya or verb')
            continue
        count = count + 1
        ma_temp =dict()
        for r in s.split('>'):
            if r:
                if len(r.split('<')) == 2:
                    ma_temp[r.split('<')[1].split(':')[0]] = transliterate(r.split('<')[1].split(':')[1], sanscript.WX, sanscript.SLP1)
                else:
                    print(r)
                    ma_temp[r.split(':')[0]] = transliterate(r.split(':')[1], sanscript.WX, sanscript.SLP1)
        [gender,case,vachan] = [ma_temp['lifgam'],ma_temp['viBakwiH'],ma_temp['vacanam']]
  
        if [gender,case,vachan] == gt:
            print('Predicted',[gender,case,vachan])
            print('Ground truth',gt)
            return count
    return -1
s = 'pANdava<vargaH:nA><lifgam:puM><viBakwiH:6><vacanam:bahu><level:1>'
# match_noun([s],'{puM}{6;bahu}')   

In [10]:
def overall_match(words,ma):
    tag = map_corpus_tag_to_dcs(ma)
    print('DCS mapped tag: ',tag)
    if tag == 'na':
        print('Tag mapping failed')
        return -1
    if dcs_to_uoh_dict[tag][0]<=71:
        print('Mapping noun')
        print('UoH Tag: ',dcs_to_uoh_dict[tag][1])
        return match_noun(words,dcs_to_uoh_dict[tag][1],ma)
    elif dcs_to_uoh_dict[tag][0]<=78:
        print('Mapping Indeclineables')
        print('UoH Tag: ',dcs_to_uoh_dict[tag][1])
        return match_Indeclineables(words,dcs_to_uoh_dict[tag][1])
    else:
        print('Mapping verb')
        print('UoH Tag: ',dcs_to_uoh_dict[tag][1])
        return match_verb(words,dcs_to_uoh_dict[tag][1])

In [11]:
import random
import os
import pandas as pd

path='./sanskrit_treebank/Graph/'
files = os.listdir(path)
ct = 0
for name in files:
#     name = 'graph_Gold_1152'
    ct = ct + 1
    print('#################################')
    print('Currentl working on ',name)
    print('Percentage: ',(ct/len(files))*100.0)
    print('#################################')
    check = name.replace('graph','rule')
    if os.path.exists('./sanskrit_treebank/Rules/'+check):
        print('File exists')
        continue
    ##############################################
    # Load graph, MA, gold file
    gold = name.replace('graph_','')+'.csv'
    df_gold = pd.read_csv('./sanskrit_treebank/Gold/'+gold,sep=',')
    try:
        df_graph = pd.read_csv('./sanskrit_treebank/Graph/'+name,sep=' ',header=None) 
    except:
        print('This graph file is empty')
        continue
    ma = 'ma_'+gold.replace('.csv','')
    df_ma = pd.read_csv('./sanskrit_treebank/MA/'+ma,sep='\t',header=None)  
    ##############################################
    
    ###############################################
    # Choose correct MA from UoH space
    choose_correct_ma = [-1]*len(df_gold)
    for i in range(len(df_gold)):  
        print('*****'+str(i)+'*******')
        print('Corpus tag:',df_gold.iloc[i,3])
        if i>=len(df_ma):
            break
        for f in df_ma.iloc[i,-1].split('/'):
            print(f)
        print(overall_match(df_ma.iloc[i,-1].split('/'),df_gold.iloc[i,3]))
        choose_correct_ma[i] = overall_match(df_ma.iloc[i,-1].split('/'),df_gold.iloc[i,3])
    ################################################
    
    #############################################
    # Remove tags other than 22 tags
    delete_rows = list()
    for key in uoh_rel_22_tags_dict.keys():
        if uoh_rel_22_tags_dict[key] == 'na':
            delete_rows.append(int(key))
    delete_rows.append(0)
    for d in delete_rows:
        df_graph = df_graph[df_graph[2] != d]

    #########################################
    # Keep only tags as per correct MA and add tag as per corpus set(22 size)
    print('List of correct MA')
    for i,rm in enumerate(choose_correct_ma):
        if rm == -1:
            continue
        print(i+1,rm)
        df_graph.drop(df_graph[(df_graph[0]==i+1) & (df_graph[1]!= rm)].index, inplace=True)
        df_graph.drop(df_graph[(df_graph[3]==i+1) & (df_graph[4]!= rm)].index, inplace=True)
    df_graph.reset_index(drop=True,inplace=True)
    for i in range(len(df_graph)):
        df_graph.iloc[i,7] = uoh_rel_22_tags_dict[str(df_graph.iloc[i,2])]
    ##########################################
 
    # Remove duplicate and save features
    df_graph = df_graph.drop_duplicates(subset=[0,3,7])
    name = name.replace('graph','rule')
    df_graph.to_csv('./sanskrit_treebank/Rules/'+name,header=None,sep=',')
#     break

#################################
Currentl working on  graph_Gold_745
Percentage:  0.030147723846849564
#################################
File exists
#################################
Currentl working on  graph_Gold_2173
Percentage:  0.06029544769369913
#################################
File exists
#################################
Currentl working on  graph_Gold_761
Percentage:  0.09044317154054868
#################################
File exists
#################################
Currentl working on  graph_Gold_1803
Percentage:  0.12059089538739826
#################################
File exists
#################################
Currentl working on  graph_Gold_309
Percentage:  0.1507386192342478
#################################
File exists
#################################
Currentl working on  graph_Gold_148
Percentage:  0.18088634308109736
#################################
File exists
#################################
Currentl working on  graph_4077
Percentage:  0.21103406692794696
#####

#################################
Currentl working on  graph_Gold_611
Percentage:  14.772384684956286
#################################
File exists
#################################
Currentl working on  graph_Gold_297
Percentage:  14.802532408803135
#################################
File exists
#################################
Currentl working on  graph_1389
Percentage:  14.832680132649987
#################################
File exists
#################################
Currentl working on  graph_3946
Percentage:  14.862827856496835
#################################
File exists
#################################
Currentl working on  graph_3893
Percentage:  14.892975580343684
#################################
File exists
#################################
Currentl working on  graph_Gold_3
Percentage:  14.923123304190533
#################################
This graph file is empty
#################################
Currentl working on  graph_Gold_1669
Percentage:  14.953271028037381
##########

Percentage:  31.986735001507387
#################################
File exists
#################################
Currentl working on  graph_Gold_1056
Percentage:  32.016882725354236
#################################
File exists
#################################
Currentl working on  graph_Gold_690
Percentage:  32.04703044920109
#################################
File exists
#################################
Currentl working on  graph_Gold_1298
Percentage:  32.07717817304793
#################################
File exists
#################################
Currentl working on  graph_Gold_1196
Percentage:  32.107325896894785
#################################
File exists
#################################
Currentl working on  graph_Gold_1817
Percentage:  32.13747362074164
#################################
File exists
#################################
Currentl working on  graph_Gold_975
Percentage:  32.16762134458848
#################################
This graph file is empty
#####################

File exists
#################################
Currentl working on  graph_1989
Percentage:  49.47241483268014
#################################
File exists
#################################
Currentl working on  graph_2409
Percentage:  49.50256255652698
#################################
File exists
#################################
Currentl working on  graph_Gold_1004
Percentage:  49.532710280373834
#################################
File exists
#################################
Currentl working on  graph_Gold_1013
Percentage:  49.562858004220686
#################################
File exists
#################################
Currentl working on  graph_229
Percentage:  49.59300572806753
#################################
File exists
#################################
Currentl working on  graph_1530
Percentage:  49.62315345191438
#################################
File exists
#################################
Currentl working on  graph_4109
Percentage:  49.65330117576123
######################

Percentage:  68.58607175158275
#################################
File exists
#################################
Currentl working on  graph_Gold_2376
Percentage:  68.6162194754296
#################################
This graph file is empty
#################################
Currentl working on  graph_Gold_668
Percentage:  68.64636719927645
#################################
File exists
#################################
Currentl working on  graph_44
Percentage:  68.6765149231233
#################################
File exists
#################################
Currentl working on  graph_Gold_411
Percentage:  68.70666264697014
#################################
File exists
#################################
Currentl working on  graph_Gold_1332
Percentage:  68.736810370817
#################################
File exists
#################################
Currentl working on  graph_Gold_1649
Percentage:  68.76695809466385
#################################
File exists
#################################
C

...

*****0*******
Corpus tag: nom. pl. *
RaR<vargaH:saMKyeyam><lifgam:a><viBakwiH:7><vacanam:bahu><level:1>
DCS mapped tag:  nom. pl. m.
Mapping noun
UoH Tag:  {puM}{1;bahu}
-1
DCS mapped tag:  nom. pl. m.
Mapping noun
UoH Tag:  {puM}{1;bahu}
*****1*******
Corpus tag: nom. pl. m.
mAsa<vargaH:nA><lifgam:puM><viBakwiH:7><vacanam:bahu><level:1>
DCS mapped tag:  nom. pl. m.
Mapping noun
UoH Tag:  {puM}{1;bahu}
-1
DCS mapped tag:  nom. pl. m.
Mapping noun
UoH Tag:  {puM}{1;bahu}
*****2*******
Corpus tag: pp.
i1<upasarga:vi_awi><kqw_prawyayaH:kwa><XAwuH:iN><gaNaH:axAxiH><kqw_pratipadika:vyawIwa><vargaH:nA><lifgam:puM><viBakwiH:7><vacanam:bahu><level:2>
i1<upasarga:vi_awi><kqw_prawyayaH:kwa><XAwuH:iN><gaNaH:axAxiH><kqw_pratipadika:vyawIwa><vargaH:nA><lifgam:napuM><viBakwiH:7><vacanam:bahu><level:2>
DCS mapped tag:  na
Tag mapping failed
-1
DCS mapped tag:  na
Tag mapping failed
*****3*******
Corpus tag: nom. sg. m.
sAXumaNdalI
DCS mapped tag:  nom. sg. m.
Mapping noun
UoH Tag:  {puM}{1;eka}
UoH a

*****0*******
Corpus tag: conj.
puraH<vargaH:avy><level:1>
pur<vargaH:nA><lifgam:swrI><viBakwiH:1><vacanam:bahu><level:1>
pur<vargaH:nA><lifgam:swrI><viBakwiH:2><vacanam:bahu><level:1>
pur<vargaH:nA><lifgam:swrI><viBakwiH:5><vacanam:eka><level:1>
pur<vargaH:nA><lifgam:swrI><viBakwiH:6><vacanam:eka><level:1>
pur<vargaH:nA><lifgam:swrI><viBakwiH:8><vacanam:bahu><level:1>
DCS mapped tag:  conj.
Mapping Indeclineables
UoH Tag:  avyaya
Predicted avy
Ground truth avyaya
1
DCS mapped tag:  conj.
Mapping Indeclineables
UoH Tag:  avyaya
Predicted avy
Ground truth avyaya
*****1*******
Corpus tag: acc. sg. n.
hariwaka<vargaH:nA><lifgam:napuM><viBakwiH:1><vacanam:eka><level:1>
hariwaka<vargaH:nA><lifgam:napuM><viBakwiH:2><vacanam:eka><level:1>
DCS mapped tag:  acc. sg. n.
Mapping noun
UoH Tag:  {napuM}{2;eka}
Predicted ['napuM', '2', 'eka']
Ground truth ['napuM', '2', 'eka']
2
DCS mapped tag:  acc. sg. n.
Mapping noun
UoH Tag:  {napuM}{2;eka}
Predicted ['napuM', '2', 'eka']
Ground truth ['napuM', 

List of correct MA
2 1
3 1
5 1
6 1
8 1
9 1
10 1
11 1
12 1
13 2
15 1
16 1
17 2
18 1
22 1
24 1
25 1
26 1
27 1
28 1
#################################
Currentl working on  graph_Gold_1869
Percentage:  99.75881820922521
#################################
*****0*******
Corpus tag: conj.
sarvawra<vargaH:avy><level:1>
DCS mapped tag:  conj.
Mapping Indeclineables
UoH Tag:  avyaya
Predicted avy
Ground truth avyaya
1
DCS mapped tag:  conj.
Mapping Indeclineables
UoH Tag:  avyaya
Predicted avy
Ground truth avyaya
*****1*******
Corpus tag: g. sg. m.
Oxarika<vargaH:nA><lifgam:puM><viBakwiH:6><vacanam:eka><level:1>
Oxarika<vargaH:nA><lifgam:napuM><viBakwiH:6><vacanam:eka><level:1>
DCS mapped tag:  g. sg. m.
Mapping noun
UoH Tag:  {puM}{6;eka}
Predicted ['puM', '6', 'eka']
Ground truth ['puM', '6', 'eka']
1
DCS mapped tag:  g. sg. m.
Mapping noun
UoH Tag:  {puM}{6;eka}
Predicted ['puM', '6', 'eka']
Ground truth ['puM', '6', 'eka']
*****2*******
Corpus tag: nom. sg. n.
aByavahArya<vargaH:nA><lifgam:puM

List of correct MA
1 1
2 1
3 1
4 1
#################################
Currentl working on  graph_3055
Percentage:  99.8794091046126
#################################
*****0*******
Corpus tag: conj.
ekaxA<vargaH:avy><level:1>
DCS mapped tag:  conj.
Mapping Indeclineables
UoH Tag:  avyaya
Predicted avy
Ground truth avyaya
1
DCS mapped tag:  conj.
Mapping Indeclineables
UoH Tag:  avyaya
Predicted avy
Ground truth avyaya
*****1*******
Corpus tag: acc. sg. m.
kavirAja<vargaH:nA><lifgam:puM><viBakwiH:2><vacanam:eka><level:1>
DCS mapped tag:  acc. sg. m.
Mapping noun
UoH Tag:  {puM}{2;eka}
Predicted ['puM', '2', 'eka']
Ground truth ['puM', '2', 'eka']
1
DCS mapped tag:  acc. sg. m.
Mapping noun
UoH Tag:  {puM}{2;eka}
Predicted ['puM', '2', 'eka']
Ground truth ['puM', '2', 'eka']
*****2*******
Corpus tag: conj.
xqS1<vargaH:avy><kqw_prawyayaH:wumun><XAwuH:xqSizr><gaNaH:BvAxiH><level:2>
DCS mapped tag:  conj.
Mapping Indeclineables
UoH Tag:  avyaya
Predicted avy
Ground truth avyaya
1
DCS mapped t

List of correct MA
1 2
2 4
3 1
#################################
Currentl working on  graph_Gold_1972
Percentage:  100.0
#################################
*****0*******
Corpus tag: loc. sg. n.
sq1<prayogaH:karwari><lakAraH:lat><puruRaH:ma><vacanam:eka><paxI:parasmEpaxI><XAwuH:sq><gaNaH:BvAxiH><level:1>
sarasI<vargaH:nA><lifgam:swrI><viBakwiH:8><vacanam:eka><level:1>
saras<vargaH:nA><lifgam:napuM><viBakwiH:7><vacanam:eka><level:1>
DCS mapped tag:  loc. sg. n.
Mapping noun
UoH Tag:  {napuM}{7;eka}
It may be avyaya or verb
Predicted ['napuM', '7', 'eka']
Ground truth ['napuM', '7', 'eka']
2
DCS mapped tag:  loc. sg. n.
Mapping noun
UoH Tag:  {napuM}{7;eka}
It may be avyaya or verb
Predicted ['napuM', '7', 'eka']
Ground truth ['napuM', '7', 'eka']
*****1*******
Corpus tag: nom. sg. m.
viwawacalacCiKaafgulIkaH
DCS mapped tag:  nom. sg. m.
Mapping noun
UoH Tag:  {puM}{1;eka}
UoH analysis in not available
-1
DCS mapped tag:  nom. sg. m.
Mapping noun
UoH Tag:  {puM}{1;eka}
UoH analysis in not 